In [1]:
# Multiple outputs per cell in Jupyter 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
%%javascript
// Evitar autoscroll.
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [25]:
# a = [1,2,3]
# b = [4,5,6]
# zipped_list = zip(a,b)
zipped_list=[(1, 4), (3, 5), (2, 6)]
zipped_list
unzipped_list=list(zip(*zipped_list))
unzipped_list
list(unzipped_list[0])
list(unzipped_list[1])

[(1, 4), (3, 5), (2, 6)]

[(1, 3, 2), (4, 5, 6)]

[1, 3, 2]

[4, 5, 6]

In [36]:
p=[(1, 4), (3, 5), (2, 6)]

p.sort()
p

p=list(zip(*p))
p

p=[list(p[0]),list(p[1])]
p

[(1, 4), (2, 6), (3, 5)]

[(1, 2, 3), (4, 6, 5)]

[[1, 2, 3], [4, 6, 5]]

In [30]:
import numpy as np
# from toolbox import *
import toolbox as tb
from skeleton import *
import pandas as pd
from gensim.models import FastText
%load_ext autoreload
%autoreload 2
import os
import tqdm
from microtc.utils import tweet_iterator
import datetime


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [59]:
class Tokens():
    tokenized_pointer=None
    
    def tokenize(self,json_file,npy_file=None,replace=False):
        self.npy_file=str(os.path.splitext(json_file)[0]+".npy") if not npy_file else npy_file

        if not replace and os.path.isfile(self.npy_file):
            print(f"** Replace is off. {os.path.abspath(self.npy_file)} already exists, then load.")
        else:
            tokenized_docs=[]
            self.N=0
            print(f"** Processing {json_file} ...")
            tx = datetime.datetime.now()
            for idx, tw in enumerate(tweet_iterator(json_file)):
                tb.show_progress(1000, tx, idx)
                twTokens = tb.process_line(tw['text']) # Limpiando text.
#                 print(type(twTokens))
                tokenized_docs.append(twTokens)
            self.N=idx+1
            print(type(tokenized_docs))
            print(tokenized_docs)

            np.save(self.npy_file,tokenized_docs)
            del(tokenized_docs)
            print(f"** Processed {self.N} lines. Saved to {os.path.abspath(self.npy_file)}.")
        
        self.tokenized_pointer=np.load(self.npy_file, mmap_mode='r')
        return self
    
    def get(s):
        return self.tokenized_pointer[s]

In [104]:
np.random.rand(2)

array([0.00156783, 0.8219782 ])

In [60]:
tokens=Tokens().tokenize("data/geo-mx-2004_min.json",replace=True)

** Processing data/geo-mx-2004_min.json ...
2020-05-12 02:14:33.949692 :: 0.000219 - Processing item #0 
<class 'list'>
[['nochecit'], ['teo', 'pai', 'pai', 'canon', 'nunc', 'crei', 'viv', 'moment', 'exist', 'graci'], ['jajajaj', 'sab', 'sup', 'chid', 'manej', 'mism', 'user', 'igual', 'amig'], ['obvi', 'hoy', 'cumpl', 'anos', 'apag', 'voz'], ['jajajajaj', 'campanaz'], ['hoy', 'cumpl', 'ano', 'plan', 'des', 'secret', 'gent', 'sabi', 'haci', 'sent', 'bien', 'aunqu', 've', 'tan', 'cercan', 'dia', 'vuelv', 'tenert', 'dej', 'recordart'], ['dij', 'amig', 'analiz', 'porn', 'tan', 'satur', 'wey', 'buenis', 'gent', 'normal', 'consider', 'product', 'nich', 'va', 'sup', 'bien'], ['dan', 'ris', 'mem', 'vat', 'tres', 'pel', 'barb', 'si', 'jajajaj', 'sal', 'pel', 'baj', 'barbil', 'lad', 'car', 'amp', 'its', 'so', 'funy'], ['si', 'domicili', 'solicit', 'oblig', 'solidari', 'mism', 'proced', 'mand', 'dos', 'comprob', 'domicili', 'diferent', 'bien'], ['amig', 'seguidor', 'tod', 'person', 'respet', 'jue

ValueError: Array can't be memory-mapped: Python objects in dtype.

In [ ]:
class Index():
#     vocab = None
    N = 0
    DF = Counter({})
    TF = []
    
#     DF_vec = None
#     IDF_vec = None

    def computePostingLists(self, filename, showProgressEach=None):
        print("** Processing " + filename + "...")
#         tx = datetime.datetime.now()
        for idx, tw in tqdm(enumerate(tweet_iterator(filename))):
#             tb.show_progress(showProgressEach,tx,idx)
            twTxtLst = tb.process_line(tw['text']) # Limpiando text.
            
            # Sumando frecuencias individiales.
            twCnt = Counter(twTxtLst)
            
            TF.append(twCnt)
            
            # Sumando frecuencias por documento (DF).
            self.DF.update(list(twCnt.keys()))

            # Contabilizando documentos.
            self.N += 1
            
#         words = self.DF.keys()
#         self.vocab = dict(zip(words, range(len(words))))
#         self.DF_vec = cnt2vec(self.DF, self.vocab)
#         self.IDF_vec = np.log2(self.N / (self.DF_vec + 1))
        
#         # Calculando TF-IDF y asignando a usuarios.
#         for userId in self.users:
#             TF_cnt = self.users[userId]
#             self.users[userId] = cnt2vec(TF_cnt, self.vocab) * self.IDF_vec
        postlists=defaultdict(list)

    def getX(self):
        return np.array([self.users[key] for key in self.users])
    
    def cluster(self, n_clusters=9):
        self.cluster = KMeans(n_clusters=n_clusters, n_jobs=4).fit(self.getX())
        return self.cluster

In [133]:
# obj=np.arange(100*50000).reshape((50000,100))
# obj=[['nochecit','nochecit'], 
#      ['teo', 'pai'], 
#      ['jajajaj', 'sab'], 
#      ['obvi', 'hoy']]
obj=[[1], 
     [3, 4], 
     [5, 6], 
     [7, 8]]
np.save("obj.npy",obj)

In [ ]:
np.memmap("obj.npy",mode='r', dtype='str')

In [135]:
np.loadtxt("obj.npy", mmap_mode='r')

TypeError: loadtxt() got an unexpected keyword argument 'mmap_mode'

In [116]:
X = np.load("obj.npy", mmap_mode='r',shape=(4,2))
X.shape
X[1,:]
# X = np.load("obj.npy")
# for i in range(20):
#     print(X[i, :])

TypeError: load() got an unexpected keyword argument 'shape'

In [10]:
import sys
local_vars = list(locals().items())
tot=0
for var, obj in local_vars:
    mem=sys.getsizeof(obj)
    tot+=mem
    print(var, mem)
print("Total =",tot)

__name__ 57
__doc__ 113
__package__ 16
__loader__ 16
__spec__ 16
__builtin__ 88
__builtins__ 88
_ih 200
_oh 248
_dh 80
In 200
Out 248
get_ipython 72
exit 64
quit 64
_ 72
__ 49
___ 49
_i 162
_ii 154
_iii 154
_i1 195
InteractiveShell 1064
_i2 167
_i3 225
np 88
tb 88
skmeans 1064
transforms 88
plt 88
cm 88
nltk 88
itertools 88
string 88
unicodedata 88
re 88
word_tokenize 144
PCA 1064
pd 88
faiss 88
axes3d 88
punctuation_table 1192
stop_words 2904
euclidiana 144
coseno 144
plotClusters 144
plotPCA 144
preprocess 144
tokenize_sentences 144
cocurrency_matrix 144
plotDecisionBoundary 144
NearestCentroid 1064
kNN 1064
Clustering 1064
FastText 1064
_i4 58
_i5 154
X 144
_i6 117
obj 112
_i7 154
_i8 154
_i9 162
_9 72
_i10 210
sys 88
Total = 18206
